In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import *
import html

In [39]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH='/mnt/samsung_1tb/Data/fastai/awdlstm/data/penn/'

In [60]:
LM_PATH='/mnt/samsung_1tb/Data/fastai/lm/models/penn/'
LM_PATH_TMP = LM_PATH+'tmp'

In [27]:
data = pd.read_csv(PATH+'train.txt', header=None)
        
LANG_TEXT = pd.DataFrame(data)

In [28]:
LANG_TEXT.head()

,0
0,aer banknote berlitz calloway centrust cluett...
1,pierre <unk> N years old will join the board ...
2,mr. <unk> is chairman of <unk> n.v. the dutch...
3,rudolph <unk> N years old and former chairman...
4,a form of asbestos once used to make kent cig...


In [30]:
LANG_TEXT.columns = [ 'text']

In [31]:
LANG_TEXT['length'] = LANG_TEXT['text'].str.len()

In [32]:
LANG_TEXT.head()

,text,length
0,aer banknote berlitz calloway centrust cluett...,176
1,pierre <unk> N years old will join the board ...,80
2,mr. <unk> is chairman of <unk> n.v. the dutch...,64
3,rudolph <unk> N years old and former chairman...,153
4,a form of asbestos once used to make kent cig...,189


In [33]:
LANG_TEXT = LANG_TEXT.sort_values(by=['length'], ascending=False)

In [34]:
len(LANG_TEXT)

42068

In [35]:
LANG_TEXT.describe()

,length
count,42068.000000
mean,120.270752
std,58.694396
min,4.000000
25%,77.000000
50%,115.000000
75%,156.000000
max,520.000000


## Language model tokens

In [36]:
chunksize = 5000

In [47]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [52]:
def get_texts(df):
    texts = f'\n{BOS} {FLD} 1 ' + df[0].astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok


In [53]:
df_trn = pd.read_csv(PATH+'train.txt', header=None)
df_val = pd.read_csv(PATH+'valid.txt', header=None)

In [62]:
tok_trn = get_texts(df_trn)

In [63]:
tok_val = get_texts(df_val)

In [65]:
np.save(LM_PATH+'tmp/tok_trn.npy', tok_trn)
np.save(LM_PATH+'tmp/tok_val.npy', tok_val)

In [66]:
tok_trn = np.load(LM_PATH+'tmp/tok_trn.npy')
tok_val = np.load(LM_PATH+'tmp/tok_val.npy')

In [67]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

[('the', 50863),
 ('<unk>', 44972),
 ('\n', 42068),
 ('xbos', 42068),
 ('xfld', 42068),
 ('1', 42068),
 ('n', 32379),
 ('of', 24406),
 ('to', 23662),
 ('a', 21253),
 ('in', 18010),
 ('and', 17498),
 ('.', 11866),
 ("'s", 9784),
 ('that', 8931),
 ('for', 8927),
 ('$', 7541),
 ('is', 7337),
 ('it', 6112),
 ('said', 6027),
 ('on', 5653),
 ('-', 5525),
 ('by', 4915),
 ('at', 4894),
 ('as', 4833)]

In [68]:
max_vocab = 60000
min_freq = 2

In [70]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

9698

In [71]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [72]:
np.save(LM_PATH+'tmp/trn_ids.npy', trn_lm)
np.save(LM_PATH+'tmp/val_ids.npy', val_lm)
pickle.dump(itos, open(LM_PATH+'tmp/itos.pkl', 'wb'))

In [73]:
trn_lm = np.load(LM_PATH+'tmp/trn_ids.npy')
val_lm = np.load(LM_PATH+'tmp/val_ids.npy')
itos = pickle.load(open(LM_PATH+'tmp/itos.pkl', 'rb'))

In [74]:
vs=len(itos)
vs,len(trn_lm)

(9698, 42068)